The Australian DEM is 35GB, and hosted by the DEA [here](https://data.dea.ga.gov.au/?prefix=projects/elevation/ga_srtm_dem1sv1_0/)
AWS bucket name to use to access the data is `dea-public-data`

## Original Data Source

*SRTM-derived 1 Second Digital Elevation Models Version 1.0*

The 1 second Shuttle Radar Topography Mission (SRTM) Digital Elevation
Models Version 1.0 package comprises three surface models: the Digital
Elevation Model (DEM), the Smoothed Digital Elevation Model (DEM-S)
and the Hydrologically Enforced Digital Elevation Model (DEM-H).

### sources and guides
[DEA guide to accessing data from their public S3 bucket using STAC](https://docs.dea.ga.gov.au/guides/setup/gis/stac/)

[Downloading and streaming data using STAC metadata](https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/)

In [172]:
import os
import warnings
import numpy as np

from osgeo import gdal

# packages for widgets and interactive map:
from ipywidgets import widgets, IntSlider, jslink, HBox, Output
from IPython.display import display, HTML, clear_output
from ipyleaflet import Map, GeoData, ImageOverlay, basemaps, LayersControl, ScaleControl, FullScreenControl, WidgetControl, LocalTileLayer
import io
import base64
import rioxarray
import xarray as xr
import xarray_leaflet
import matplotlib.pyplot as plt
import gdal2tiles

from localtileserver import get_leaflet_tile_layer, TileClient, Report

from rio_tiler.io import Reader
from rio_tiler.models import ImageData

#packages for geo-processing:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds
import xarray as xr
import xarray_leaflet

import pystac_client
import stackstac
#import odc.stac
import pyproj
from dask.distributed import Client, LocalCluster
import pandas as pd

In [173]:
with Reader('test-dem.tif') as src:
    info = src.info()
    stats = src.statistics()
    data = src.preview()

print(stats['b1'].max)
print(data.data.shape)

max_height = stats['b1'].max

rescale = data.post_process(
    in_range=((0, max_height),),
    out_range = ((0,255),)
)

print(rescale.data.dtype)

buf = rescale.render(img_format='png')


421.1336669921875
(1, 313, 458)
uint8


/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)


## environment and STAC setup

In [174]:
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
client = Client()

cm = cmap.get("viridis")

"""
If STAC service is down, try using direct link to the DEM-H geotiff file
"""
dem_href = 'https://dea-public-data.s3-ap-southeast-2.amazonaws.com/projects/elevation/ga_srtm_dem1sv1_0/dem1sv1_0.tif'


#catalog_dem = pystac_client.Client.open("https://explorer.sandbox.dea.ga.gov.au/stac/")
#collection_dem = ['ga_srtm_dem1sv1_0']

crs_string = "epsg:3857"
epsg = pyproj.CRS.from_string(crs_string).to_epsg()
resolution = 90 #added to resolve error "Failed to auto-guess CRS/resolution."


/opt/conda/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42549 instead
  warnings.warn(


## interactive map so users can upload aoi files

In [175]:
#upload button
uploader = widgets.FileUpload(
    description='Upload file',
    accept='.geojson',
    multiple=False,
    button_style='primary'
)

#submit polygon button
submit = widgets.Button(
    description='Display AOI', 
    button_style='success', 
    tooltip='Click me')

#data processing/fetching button
get_data = widgets.Button(
    description='Get data for AOI', 
    button_style='primary', 
    tooltip='Click me')


 #note that the center coordinates are the opposite to what is stored in a geojson, you need to swap the lat/long
m = Map(height='500px', width='500px', 
          center=(-29.2253, 116.26012), zoom=10,
          basemap = basemaps.OpenTopoMap)


In [176]:
def get_vector(upload_widget):
    try:
        uploaded_filename = next(iter(upload_widget.value))
        content = upload_widget.value[uploaded_filename]['content']
        bytes_io = io.BytesIO(content)
        vector = gpd.read_file(bytes_io)
        print(vector)

    except Exception as e:
        print(e)
    return vector


def submit_clicked(b):
    try:
        fc = get_vector(uploader)
        geo_json = GeoData(geo_dataframe = fc, name='AOI')
        m.add_layer(geo_json)
    except Exception as e:
        print(e)
    

## using uploaded aoi to get DEM

In [177]:

def get_data_clicked(b):
    fc = get_vector(uploader)
    bbox = list(fc.total_bounds)
     
    with rasterio.open(dem_href) as src:
        window=from_bounds(*bbox, transform=src.transform)
        data = src.read(window=window)
        kwargs = src.meta.copy()
        kwargs.update({
        'height': window.height,
        'width': window.width,
        'transform': rasterio.windows.transform(window, src.transform)})
    
    data_scaled = data/255.0
    
    with rasterio.open('test-dem-scaled.tif', 'w', **kwargs) as dst:
            dst.write(data_scaled)
    

    content_b64 = b64encode(data.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))
       
    gdal2tiles.generate_tiles("test-dem-scaled.tif", "tileset")

    test_dem = rioxarray.open_rasterio('test-dem-scaled.tif') 
    test_dem = test_dem.sel(band=1)
    da = test_dem.rio.write_crs(4326)  # WGS 84
    da = da.rio.write_nodata(np.nan)
    da.leaflet.plot(m, colormap=plt.cm.viridis)  
    
    layer = CustomLocalLayer(tms=True, path="tileset/{z}/{x}/{y}.png")
    m.add_layer(layer)
    
    return payload
    


In [178]:
buttons = HBox([uploader, submit, get_data])
display(buttons)

download_output = Output()
display(download_output)


# zoom slider and widget control
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=10)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widgetControl = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widgetControl)

m.add_layer(basemaps.OpenStreetMap.Mapnik)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl(position='topright'))
m.add_control(LayersControl())
display(m)

    
submit.on_click(submit_clicked)
get_data.on_click(get_data_clicked)

Output()

Map(center=[-29.2253, 116.26012], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Download the fetched DEM raster:

In [179]:
# import tempfile
# test_dem = None
# # FILEPATH: /workspaces/data-notebooks/notebooks/evergreen-resources/get-elevation-data-for-aoi.ipynb
# #this cell is for the get_data button

# def create_download_link(output_filename, title="Download GeoTIFF", filename="data.tif"):
#     data = open(output_filename, "rb").read()
#     b64 = base64.b64encode(data).decode()

#     html = f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">{title}</a>'
#     return HTML(html)

# def get_data(b):
#     global test_dem
#     fc = get_vector(uploader) #don't use the geoData global variable, use the geodataframe
#     bbox = list(fc.total_bounds)
#     ds_dem = catalog_dem.search(
#     bbox=bbox, 
#     collections=collection_dem)

#     items = ds_dem.item_collection()
#     bands_of_interest = 'dem_h'

#     da = stackstac.stack(
#         items,
#         assets=[bands_of_interest],
#         epsg=epsg,
#         resolution=resolution,
#         bounds_latlon=bbox# <-- use Dask
#     ).isel(time=0)
#     da = da.compute()
#     test_dem = da
    
#     #da.leaflet.plot(Map, colormap=plt.cm.terrain)
    
#     # output_filename = "clipped_raster.tif"
#     # with rasterio.open(
#     #     output_filename,
#     #     'w',
#     #     driver='GTiff',
#     #     height=da.shape[1],
#     #     width=da.shape[2],
#     #     count=da.shape[0],
#     #     dtype=da.dtype,
#     #     crs=da.crs,
#     #     transform=da.transform,
#     # ) as dst:
#     #     dst.write(da)
        
#     # temp_raster = rioxarray.open_rasterio(output_filename)
#     # temp_raster_4326 = temp_raster.rio.reproject('EPSG:4326')
#     # temp_raster_4326_mapband = temp_raster_4326.sel(band=1)

#     # temp_raster_4326_mapband.leaflet.plot(Map, colormap=plt.cm.terrain)

#     #download_button = create_download_link(output_filename, title="Download GeoTIFF", filename="demdata.tif")

#     return da
    



## This section of code uses a hardcoded geojson to test. Remove once widget version is working.

Search for DEM data in the STAC catalog. Note that there is no timeframe filtering here because there is only 1, perpetual DEM in this catalog. If you add a timeframe you will get 0 results.
There are 3 assets in this collection: `dem`, `dem_s` and `dem_h`. All three are based on the NASA SRTM elevation data, so just use `dem_h`.
We will also get the url for the `dem_h` COG, so we can display it on a leafmap later on.

In [180]:
# ds_dem = catalog_dem.search(
#     bbox=bbox, 
#     collections=collection_dem)

# items = ds_dem.item_collection()
# print(f"Found {len(items)} items in the {collection_dem}")

In [181]:
# first_item = items.items[0]
# all_bands = list(first_item.assets.keys())
# print("Assets available:")
# print(*all_bands, sep=', ')

In [182]:
# bands_of_interest = 'dem_h'

# da = stackstac.stack(
#     items,
#     assets=[bands_of_interest],
#     epsg=epsg,
#     resolution=resolution,
#     bounds_latlon=bbox# <-- use Dask
# ).isel(time=0)
# da

Stack the DEM data, specifying the spatial bounds and desired resolution

One thing to watch out for with stackstac.stack is that you will wind up with a distinct time coordinate for each STAC item that you pass in. To achieve the intuitive representation of the data, you need to flatten the DataArray with respect to day.
Note: if you are only reading a single STAC item, stackstac.mosaic will inadvertently reduce your data along the band dimension (which is definitely not what you want!), hence the conditional statement checking for more than one time coordinate value.
[from here](https://hrodmn.dev/posts/stackstac/)

Here we are getting around the problem by using `isel` to select the first time epoch, as there is only one. A different approach will be needed if working with data across timeframes.

In [183]:
#da = da.compute()
# da.plot()

Save the selected DEM data as a GeoTIFF file

In [184]:
# with rasterio.open(dem_href) as dem_data:
#         # Read a window of data from the band
#         data = dem_data.read(window=from_bounds(*bbox, transform=dem_data.transform))
#         data = dem_data.read()



In [185]:

# output_filename = "data/temp/clipped_raster.tif"
# with rasterio.open(
#     output_filename,
#     'w',
#     driver='GTiff',
#     height=test_dem.shape[0],
#     width=test_dem.shape[1],
#     count=test_dem.shape[0],
#     dtype=test_dem.dtype,
#     crs=test_dem.crs,
#     transform=test_dem.transform,
# ) as dst:
#     dst.write(test_dem)

In [186]:
# # Define the filename for the reprojected raster
# reprojected_filename = "data/temp/reprojected_raster.tif"

# # Open the original clipped raster
# with rasterio.open(output_filename) as src:
#     # Define the transform and dimensions for the new raster
#     transform, width, height = calculate_default_transform(
#         src.crs, 'EPSG:4326', src.width, src.height, *src.bounds)
    
#     # Define the metadata for the new raster
#     kwargs = src.meta.copy()
#     kwargs.update({
#         'crs': 'EPSG:4326',
#         'transform': transform,
#         'width': width,
#         'height': height
#     })

#     # Create and write the reprojected raster
#     with rasterio.open(reprojected_filename, 'w', **kwargs) as dst:
#         for i in range(1, src.count + 1):
#             reproject(
#                 source=rasterio.band(src, i),
#                 destination=rasterio.band(dst, i),
#                 src_transform=src.transform,
#                 src_crs=src.crs,
#                 dst_transform=transform,
#                 dst_crs='EPSG:4326',
#                 resampling=Resampling.nearest)